In [3]:
from __future__ import unicode_literals
import requests
import json
import time
import codecs
import sys
import pandas
import datetime
from time import gmtime, strftime
UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)

In [4]:
def get_group_results(params):
    request = requests.get("http://api.meetup.com/2/groups",params=params)
    data = request.json()
    return data

In [5]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz  

def date_conversion(date):
    utc = pytz.utc  
    date = date/1000
    utc_dt =utc.localize(datetime.utcfromtimestamp(date))
    return(utc_dt.strftime('%Y-%m-%d %H:%M:%S'))

In [7]:
def get_group_data():
        data =[]
        cities =[("Atlanta","GA")]

        #api_key= 
        # Get your key here https://secure.meetup.com/meetup_api/key/
        for (city, state) in cities:
            per_page = 200
            results_we_got = per_page
            offset = 0
            while (results_we_got == per_page):
                # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
                response=get_group_results({"sign":"true",
                                      "country":"US",
                                      "city":city,
                                      "state":state,
                                      "radius": 10,
                                      "key":api_key,
                                      "page":per_page,
                                      "offset":offset })
                time.sleep(1)
                offset += 1
                results_we_got = response['meta']['count']
                for group in response['results']:
                    category = ""
                    if "category" in group:
                        category = group['category']['name']
                    created =date_conversion(group['created'])   
                    data.append([city, group['name'].replace(","," "), 
                                         created, group['city'],
                                         group.get('state',""),category,group['members'], 
                                         group.get('who',"").replace(","," ")])

                   
            time.sleep(1)
            data =pandas.DataFrame(data)
            return (data)

In [8]:
def get_member_results(params):
    request = requests.get("http://api.meetup.com/2/members",params=params)
    data = request.json()
    request.close
    return data

In [9]:
def get_member_data():
        members =[]
        groups =[("python-atlanta")]
        #api_key= 
        for group_urlname  in groups:
            per_page = 200
            results_we_got = per_page
            offset = 0
            while (results_we_got == per_page):
                response=get_member_results({"sign":"true",
                                      "group_urlname" : group_urlname,
                                      "key":api_key,
                                      "page":per_page,
                                      "offset":offset })
                time.sleep(1)
                offset += 1
                results_we_got = response['meta']['count']
                for group in response['results']:
                    for i in group['topics']:
                        #members.append
                        joined =date_conversion(group['joined']) 
                        visited =date_conversion(group['visited']) 
                        members.append(
                        [joined,group['id'],visited,
                        i['urlkey'], i['name'],i['id'],
                        group['status'],group['city'],group['name']])
                        
                        
        time.sleep(1)
        members = pandas.DataFrame(members)
        return (members)

In [7]:
def get_ga_data():
        data =[]
        api_key= 
        # Get your key here https://secure.meetup.com/meetup_api/key/
        zipz={"30009","30060","30101","30150","30151","30152","30153","30154","30156","30157","39885",
              "39886","39897","39901"}
        count=0
        for zips  in zipz:
            count+=1
            per_page = 200
            results_we_got = per_page
            offset = 0
            while (results_we_got == per_page):
                    # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
                    response=get_group_results({"sign":"true",
                                          "country":"United States",
                                          "state":"Georgia",
                                          "radius": 50,
                                          "zip" :zips,
                                          "key":api_key,
                                          "page":per_page,
                            "offset":offset })
                    #print(response)
                    time.sleep(1)
                    offset += 1
                    results_we_got = response['meta']['count']
                    for group in response['results']:
                        #for j in group["category"]:
                        #    name= group["category"]["name"]
                        #    idd = group["category"]["id"]
                        #    shortname= group["category"]["shortname"]
                        for i in group['topics']:
                            created =date_conversion(group['created']) 
                            data.append(
                            [group['urlname'],created,group['members'],group['rating'],group['join_mode'],
                             group['state'],
                             group['organizer']['member_id'],group['organizer']['member_id'],
                             group['id'],
                             #name,idd,shortname,
                            i['urlkey'], i['name'],i['id'],
                            group['timezone'],group['city'],group['visibility']
                            ])
            print(count)
            #print("Zip Abstraction over")
        data =pandas.DataFrame(data)
        print("Dataframe over")
        return (data)

In [14]:
#if __name__=="__main__":
        #group =get_group_data()
        #group.columns= ['city', 'group','created','city', 'state','category','members','who']
        #member=get_member_data()
        #member.columns =['joined','id','visited','groupkey', 'groupname','groupid',
        #        'group_status','group_city','member_name']
        #ga_groups =get_ga_data()

In [16]:
#ga_groups.drop_duplicates()


In [2]:
ga_groups.shape